In [1]:
import numpy as np
import pandas as pd
import tensorflow
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
data = pd.read_csv('wine_red_scaled.csv')

# Oddzielenie kolumny, która będzie przewidywana (quality) od reszty danych
X = data.drop('value', axis=1).values
y = data['value'].values

# Konwertowanie kategorii na zmienne binarne (dummy variables)
y = pd.get_dummies(y).values

In [3]:
def create_model(input_dim, output_dim, n_layers, n_neurons, activation):
    
    # Tworzenie nowego modelu sekwencyjnego (warstwy ułożone liniowo)
    model = tensorflow.keras.models.Sequential()
    
    # Dodanie warstwy wejściowej
    model.add(Input(shape=(input_dim,)))
    
    # Dodanie warstw ukrytych
    for _ in range(n_layers):
        model.add(Dense(n_neurons, activation=activation))
        
    # Warstwa wyjściowa, w klasyfikacji wieloklasowej softmax przekształca wyjścia na prawdopodobieństwa, które sumują się do 1 
    model.add(Dense(output_dim, activation='softmax'))
    
    # Przygotowanie modelu do procesu uczenia
    # optimizer - w jaki sposób optymalizowane są wagi, adam - adaptive moment estimation
    # loss - określa funkcję straty (miara błędu, którą model próbuje zminimalizować podczas uczenia) categorical_crossentropy oblicza logarytmiczną stratę, uwzględniając prawdopodobieństwo przewidywane dla każdej klasy
    # metrics - śledzone metryki w czasie treningu
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
param_grid = {
    "n_layers": [1, 2, 3, 5],  
    "n_neurons": [5, 10, 20, 40], 
    "activation": ['relu', 'linear', 'tanh', 'sigmoid'],
    "test_size": [0.1, 0.2, 0.3, 0.5]
}

results = []

epochs = 10

for n_layers in param_grid['n_layers']:
    for n_neurons in param_grid['n_neurons']:
        for activation in param_grid['activation']:
            for test_size in param_grid['test_size']:

                # Podział danych
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                                    stratify=y)

                # Tworzenie modelu
                model = create_model(input_dim=X.shape[1], output_dim=y.shape[1],
                                     n_layers=n_layers, n_neurons=n_neurons, activation=activation)
                
                # Trenowanie modelu
                # epochs - liczba iteracji modelu przez zbiór treningowy
                # Model przechodzi przez te dane treningowe wielokrotnie, aby nauczyć się lepiej dopasowywać do wzorców w danych
                # batch_size - podział zbioru treningowego na mniejsze próbki o tym rozmiarze,
                # przetwarzane na raz podczas aktualizacji wag
                # verbose = 0 - brak komunikatów podczas trenowania, jedynie pasek postępu
                model.fit(X_train, y_train, epochs=epochs, batch_size=16, verbose=0)

                # Przewidywanie wyników na danych testowych
                # predict przewiduje, argmax wybiera największe prawdopodobieństwo (klasę która "wygrała")
                # Przewidywanie wyników na danych uczących
                y_pred = np.argmax(model.predict(X_train), axis=1)
                y_true = np.argmax(y_train, axis=1) # pobiera prawdziwe klasy

                
                 # average = 'weighted' odpowiada za średnią ważoną,
                 # z powodu wieloklasowej klasyfikacji
                precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
                recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
                f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

                # Zapis wyników
                results.append({
                    'n_layers': n_layers,
                    'n_neurons': n_neurons,
                    'activation': activation,
                    'test_size': test_size,
                    'precision': precision,
                    'recall': recall,
                    'f1_score': f1
                })

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
40/40 ━━━━━━━━━━━━━━━━━━━

In [9]:
# Stworzenie DataFrame z wyników
results_df = pd.DataFrame(results)

# Analiza wyników
print("\nWyniki dla różnych konfiguracji:")
# Grupowanie wyników i obliczanie średnich dla accuracy i precision
print(results_df.groupby(['n_layers', 'n_neurons', 'activation', 'test_size'])[['precision', 'recall', 'f1_score']].mean())

# Zapis wyników do pliku EXCEL
results_df.to_excel('wine_uczace3.xlsx', index=False)


Wyniki dla różnych konfiguracji:
                                         precision    recall  f1_score
n_layers n_neurons activation test_size                               
1        5         linear     0.1         0.781559  0.820709  0.784900
                              0.2         0.796637  0.824863  0.810484
                              0.3         0.737092  0.819482  0.750126
                              0.5         0.782634  0.827284  0.796161
                   relu       0.1         0.680424  0.824878  0.745720
...                                            ...       ...       ...
5        40        sigmoid    0.5         0.680264  0.824781  0.745584
                   tanh       0.1         0.680424  0.824878  0.745720
                              0.2         0.680399  0.824863  0.745699
                              0.3         0.680367  0.824844  0.745672
                              0.5         0.680264  0.824781  0.745584

[256 rows x 3 columns]
